IA & Data science (LU3IN0226) -- 2024-2025
--------
*&copy; Equipe pédagogique: Christophe Marsala, Olivier Schwander, Jean-Noël Vittaut.*


# TD-TME 6 : Apprentissage pour le texte

<font size="+1" color="RED"><b>[Q]</b></font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

ZHANG Yuxiang & LECOMTE Antoine

<font color="RED" size="+1"><b>[Q]</b></font> **Renommer ce notebook**

Tout en haut de cette page, cliquer sur <tt>tme-06</tt> et rajouter à la suite de <tt>tme-06</tt> les noms des membres du binômes séparés par un tiret.

<font color="RED" size="+1">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *tme-06-Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: il faut rendre le fichier ipython tel quel, éventuellement, si vous avez d'autres fichiers vous les rendez séparément.

**Echancier pour la soumission de votre compte-rendu:**
- le compte-rendu d'une séance doit être remis obligatoirement <font color="RED">avant la séance suivante</font>.

**Le compte-rendu est soumis sur la page Moodle.**

In [1]:
# - - - - - - - - - - - - - - - - - -
# imports utiles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mtpl
%matplotlib inline  

import math
import time
import sys

# Les instructions suivantes sont utiles pour recharger automatiquement 
# le code modifié dans les librairies externes
%load_ext autoreload
%autoreload 2

# - - - - - - - - - - - - - - - - - -
# Information sur l'environnent utilisé ici:
print("Version python et des librairies:")
print("\tPython ",sys.version)
print("\tpandas: ",pd.__version__)
print("\tnumpy: ",np.__version__)
print("\tmatplotlib: ",mtpl.__version__)

Version python et des librairies:
	Python  3.9.6 (default, Nov 11 2024, 03:15:38) 
[Clang 16.0.0 (clang-1600.0.26.6)]
	pandas:  2.2.3
	numpy:  2.0.2
	matplotlib:  3.9.4


In [2]:
# Importation de votre librairie iads:
# La ligne suivante permet de préciser le chemin d'accès à la librairie iads
import sys
sys.path.append('../')   # iads doit être dans le répertoire père du répertoire courant !

# Importation de la librairie iads
import iads as iads

# importation de Classifiers
from iads import Classifiers as classif

# importation de utils
from iads import utils as ut

# importation de evaluation
from iads import evaluation as ev



# Objectifs de ce TME

<div class="alert alert-block alert-warning">
Ce TME a pour but d'implémenter des fonctions pour traiter un corpus textuel et un nouvel algorithme d'apprentissage vu dans le cours 6. 

Pour expérimenter, on utilise la base `SMS spam Collection` qui contient 5572 messages associés à 2 labels: `spam` et `ham`. 
</div>

In [3]:
# Chargement du dataset

df_spam = pd.read_csv('data/spam.csv', sep='\t', encoding = 'latin1')
df_spam

,label,message
0,ham,Go until jurong point crazy.. Available only ...
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,Nah I don't think he goes to usf he lives aro...
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,Pity * was in mood for that. So...any other s...
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
# Valeurs du label:
df_spam['label'].unique()

array(['ham', 'spam'], dtype=object)

In [5]:
# On met les labels dans une liste car cela sera utile:
les_labels = df_spam['label'].unique()

<font color="RED" size="+1"><b>[Q]</b></font> En utilisant `value_counts` (voir la doc de la librairie pandas), afficher le nombre d'exemples de chaque classe dans la base.

In [6]:
for label in les_labels:
    print(f"Nombre d'exemples pour '{label}': {df_spam['label'].value_counts()[label]}")

Nombre d'exemples pour 'ham': 4825
Nombre d'exemples pour 'spam': 747


## Prétraitements

### Nettoyage des données

Pour pouvoir travailler sur les messages, on commence par les nettoyer : il faut enlever les caractères de ponctuations, les articles, les mots trop courants, etc. (voir le cours 6).

Pour traiter les données textuelles (colonne `message` du dataset), on utilise les fonctions de la librairie `string`.

In [7]:
import string

print("Caractères de ponctuations : ", string.punctuation)

print("Mise en minuscules (pour homogénéiser l'écriture : ", "May the Force be with you!".lower())

print("Découper une phrase avec espace (retourne une liste):", "LU3IN026 est l'UE d'IA et Sciences des données.".split())

print("Découper une phrase avec apostrophe (retourne une liste):", "LU3IN026 est l'UE d'IA et Sciences des données.".split("'"))



Caractères de ponctuations :  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Mise en minuscules (pour homogénéiser l'écriture :  may the force be with you!
Découper une phrase avec espace (retourne une liste): ['LU3IN026', 'est', "l'UE", "d'IA", 'et', 'Sciences', 'des', 'données.']
Découper une phrase avec apostrophe (retourne une liste): ['LU3IN026 est l', 'UE d', 'IA et Sciences des données.']


In [8]:
# Avec le dataframe du dataset:
print("Premier message du train: ",df_spam['message'].iloc[0])

print("Résultat d'un découpage: ",df_spam['message'].iloc[0].split())

Premier message du train:  Go until jurong point  crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Résultat d'un découpage:  ['Go', 'until', 'jurong', 'point', 'crazy..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'Cine', 'there', 'got', 'amore', 'wat...']


In [9]:
# Certains éléments d'une phrase ne sont pas utiles pour le traitement, par exemple en anglais:
mots_inutiles = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `nettoyage` qui prend une chaîne de caractères et rend la chaîne après nettoyage: 1) mise en minuscules 2) remplacement des caractères de ponctuation par un espace (SAUF l'apostrophe qui doit rester car elle va être utilisée pour les mots inutiles).


In [10]:
import re

def nettoyage(chaine, stopwords=None):
    # Mise en minuscules
    chaine = chaine.lower()
    
    # Remplacement de la ponctuation sauf l'apostrophe par un espace
    ponctuation_sans_apostrophe = string.punctuation.replace("'", "")
    chaine = re.sub(f"[{re.escape(ponctuation_sans_apostrophe)}]", " ", chaine)
    
    # Découpage en mots
    mots = chaine.split()
    
    # Suppression des mots inutiles s'ils sont fournis
    if stopwords:
        mots = [mot for mot in mots if mot not in stopwords]
    
    return " ".join(mots)

In [11]:
nettoyage("LU3IN026 est l'UE d'IA et Sciences des données.")

"lu3in026 est l'ue d'ia et sciences des données"

In [12]:
nettoyage(df_spam['message'].iloc[0])

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `text2vect` qui prend une chaîne de caractères ainsi qu'une liste de mots inutiles et rend la liste composée par les mots de cette chaîne obtenus, après son nettoyage et après avoir enlevé les mots inutiles.



In [13]:
def text2vect(chaine, stopwords=None):
    # Mise en minuscules
    chaine = chaine.lower()
    
    # Remplacement de la ponctuation (sauf l'apostrophe) par un espace
    ponctuation_sans_apostrophe = string.punctuation.replace("'", "")
    chaine = re.sub(f"[{re.escape(ponctuation_sans_apostrophe)}]", " ", chaine)
    
    # Découpage en mots
    mots = chaine.split()
    
    # Suppression des mots inutiles si une liste est fournie
    if stopwords:
        mots = [mot for mot in mots if mot not in stopwords]
    
    return mots

In [14]:
text2vect("May the Force be with you!",mots_inutiles)

['may', 'force']

In [15]:
text2vect("You shan't pass!",mots_inutiles)

['pass']

In [16]:
text2vect(df_spam['message'].iloc[0],mots_inutiles)

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

<font color="RED" size="+1"><b>[Q]</b></font> Ajouter une nouvelle colonne de nom `les_mots` au dataframe `df_spam` pour laquelle chaque ligne contient le résultat de l'application de `text2vect` sur le message de l'exemple correspondant.


In [17]:
# A COMPLETER 

# Application de la fonction text2vect à chaque message
df_spam['les_mots'] = df_spam['message'].apply(lambda x: text2vect(x, mots_inutiles))

# Affichage du dataframe mis à jour
df_spam

,label,message,les_mots
0,ham,Go until jurong point crazy.. Available only ...,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,Nah I don't think he goes to usf he lives aro...,"[nah, think, goes, usf, lives, around, though]"
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,"[2nd, time, tried, 2, contact, u, u, å£750, po..."
5568,ham,Will Ì_ b going to esplanade fr home?,"[ì, b, going, esplanade, fr, home]"
5569,ham,Pity * was in mood for that. So...any other s...,"[pity, mood, suggestions]"
5570,ham,The guy did some bitching but I acted like i'd...,"[guy, bitching, acted, like, i'd, interested, ..."


### Découpage du dataset en 2 ensembles train et test

<font color="RED" size="+1"><b>[Q]</b></font> Pour mettre au point nos fonctions dans ce TME, on va travailler sur une partie du dataset, par exemple, on peut prendre 5% des exemples (en respectant la distribution des classes, donc en prenant 5% d'exemples de chaque label).
Compléter le code ci-dessous pour définir les 2 variables `df_train` et `df_test` qui contiendront chacune 1 dataframe, partie de `df_spam`. Pour construire `df_train` on prend aléatoirement 5% des exemples de chaque classe, comme on l'a déjà fait dans un TME précédent (cf. `np.random.shuffle`).


In [18]:
np.random.seed(42)  # Pour reproductibilité

# Pourcentage d'exemples de chaque classe à garder :
taux = 0.05  # 5%

# Déclaration des variables qui seront initialisées dans la boucle :
df_train = pd.DataFrame()  
df_test = pd.DataFrame()

for l in les_labels:
    nb_total = df_spam['label'].value_counts()[l]  # Nombre total d'exemples pour la classe l
    nb_pris = int(nb_total * taux)  # Nombre d'exemples à prendre pour l'entraînement
    print(f"Nombre d'exemples du label {l} pris pour apprendre: {nb_pris}")

    # Récupération des indices des exemples de cette classe
    les_ids = df_spam[df_spam['label'] == l].index.to_list()
    
    # Mélanger les indices aléatoirement
    np.random.shuffle(les_ids)

    # Séparer les indices pour l'entraînement et le test
    ids_train = les_ids[:nb_pris]  # 5% pour l'entraînement
    ids_test = les_ids[nb_pris:]   # Le reste pour le test

    # Ajouter les données correspondantes aux DataFrames
    df_train = pd.concat([df_train, df_spam.loc[ids_train]], ignore_index=True)
    df_test = pd.concat([df_test, df_spam.loc[ids_test]], ignore_index=True)

# Résultat :
print(f"Dimension de df_train:\t{df_train.shape}")
print(f"Dimension de df_test:\t{df_test.shape}")

Nombre d'exemples du label ham pris pour apprendre: 241
Nombre d'exemples du label spam pris pour apprendre: 37
Dimension de df_train:	(278, 3)
Dimension de df_test:	(5294, 3)


In [19]:
df_train

,label,message,les_mots
0,ham,I am late so call you tomorrow morning.take ca...,"[late, call, tomorrow, morning, take, care, sw..."
1,ham,U r too much close to my heart. If u go away i...,"[u, r, much, close, heart, u, go, away, shatte..."
2,ham,Wait &lt;#&gt; min..,"[wait, lt, gt, min]"
3,ham,Can you call me plz. Your number shows out of ...,"[call, plz, number, shows, coveragd, area, urg..."
4,ham,MAYBE IF YOU WOKE UP BEFORE FUCKING 3 THIS WOU...,"[maybe, woke, fucking, 3, problem]"
...,...,...,...
273,spam,This is the 2nd time we have tried 2 contact u...,"[2nd, time, tried, 2, contact, u, u, 750, poun..."
274,spam,3 FREE TAROT TEXTS! Find out about your love l...,"[3, free, tarot, texts, find, love, life, try,..."
275,spam,Do you want a new Video phone? 600 anytime any...,"[want, new, video, phone, 600, anytime, networ..."
276,spam,Had your mobile 10 mths? Update to latest Oran...,"[mobile, 10, mths, update, latest, orange, cam..."


## Calculs de probabilités

À partir d'ici on va travailler sur les données d'apprentissage (`df_train`) pour déterminer les probabilités qui serviront à classer les messages.

<font color="RED" size="+1"><b>[Q]</b></font> Construire la liste de tous les mots présents dans tous les vecteurs de `df_train`, chaque mots doit n'apparaître qu'une seule fois dans la liste obtenue. Cette liste sera stockée une fois triée dans la variable `index_mots` (et on l'appelle par la suite "index de mots").


In [20]:
index_mots = sorted(set(word for mots in df_train['les_mots'] for word in mots))


In [21]:

print("Nombre de mots trouvés: ", len(index_mots))
print("Les 10 premiers :", index_mots[0:10]) 

# pour contrôler:
for i in range(30,len(index_mots),100):
    print("\ten position ",i," --> ", index_mots[i])


Nombre de mots trouvés:  1377
Les 10 premiers : ["'", "'t", '0', '00', '01223585334', '0207', '03', '07801543489', '0800', '08000930705']
	en position  30  -->  10p
	en position  130  -->  al
	en position  230  -->  brain
	en position  330  -->  considering
	en position  430  -->  emerging
	en position  530  -->  go
	en position  630  -->  interesting
	en position  730  -->  lucky
	en position  830  -->  next
	en position  930  -->  porn
	en position  1030  -->  screaming
	en position  1130  -->  sub
	en position  1230  -->  tv
	en position  1330  -->  workin


Chaque message va maintenant être représenté comme un vecteur de valeurs 0 ou 1: ce vecteur possède autant de colonnes qu'il y a de mots dans `index_mots`. Ce vecteur est construit ainsi: pour un exemple $i$, la valeur de la colonne $j$ du vecteur vaudra 1 si la liste de mots de l'exemple $i$ contient le mot en position $j$ dans `index_mots`, ou 0 dans le cas contraire.

*Remarque:* même si un mot apparaît plusieurs fois dans la liste, cela ne compte que pour 1.

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `df2array` qui prend un dataframe `df` contenant la colonne `les_mots` ainsi qu'un index de mots et rend le `np.array` correspondant aux vecteurs de valeurs représentant les exemples de `df`. Les mots de `les_mots` qui ne sont pas dans l'index de mots ne sont pas pris en compte.  


In [22]:
def df2array(df, index_mots):
    """
    Convertit un DataFrame en un tableau NumPy binaire basé sur l'index de mots.
    
    Args:
        df (pd.DataFrame): DataFrame contenant une colonne 'les_mots' (liste de mots par message).
        index_mots (list): Liste triée des mots uniques utilisés comme index.

    Returns:
        np.array: Matrice binaire (n_samples, n_mots) où chaque ligne représente un message.
    """
    # Création d'une matrice de zéros de taille (nombre d'exemples, taille du vocabulaire)
    matrice = np.zeros((len(df), len(index_mots)), dtype=int)
    
    # Construction d'un dictionnaire pour accéder rapidement aux indices des mots
    index_dict = {mot: i for i, mot in enumerate(index_mots)}
    
    # Remplissage de la matrice
    for i, mots in enumerate(df['les_mots']):
        for mot in mots:
            if mot in index_dict:  # Vérifie si le mot est dans l'index
                matrice[i, index_dict[mot]] = 1  # Marque la présence du mot
    
    return matrice


In [23]:
mat_train = df2array(df_train,index_mots)

In [24]:
mat_train.shape

(278, 1377)

In [25]:
print("Message 0:", df_train['les_mots'].iloc[0])

print("\nPositions non nulles dans le vecteur d'index:")

for i in range(0, len(mat_train[0])):
    if mat_train[0][i] == 1:
        print("\tcolonne ",i," : ", index_mots[i])
    

Message 0: ['late', 'call', 'tomorrow', 'morning', 'take', 'care', 'sweet', 'dreams', 'u', 'ummifying', 'bye']

Positions non nulles dans le vecteur d'index:
	colonne  245  :  bye
	colonne  248  :  call
	colonne  265  :  care
	colonne  412  :  dreams
	colonne  679  :  late
	colonne  791  :  morning
	colonne  1150  :  sweet
	colonne  1154  :  take
	colonne  1209  :  tomorrow
	colonne  1238  :  u
	colonne  1244  :  ummifying


In [26]:
# on se rappelle qu'il est possible d'extraire les vecteurs correspondant à un label donné, par exemple:
len(mat_train[df_train['label']=='ham'])

241

<font color="RED" size="+1"><b>[Q]</b></font> Construire un dictionnaire qui donne, pour chaque label $l$, pour chaque mot $m$ de l'index des mots, la fréquence de $m$ parmi les exemples de `df_train` qui ont le label $l$.


*Remarque*: penser à une solution dans boucle for sur les exemples...


In [27]:
# Initialisation du dictionnaire de fréquences
frequences = dict()

# Boucle sur chaque label ('ham', 'spam')
for l in les_labels:
    # Initialisation des fréquences des mots pour le label l
    frequences[l] = [0] * len(index_mots)
    
    # Sélection des exemples avec le label l
    exemples_label_l = df_train[df_train['label'] == l]
    
    # Boucle sur chaque exemple du label l
    for _, row in exemples_label_l.iterrows():
        mots_message = set(row['les_mots'])
        for mot in mots_message:
            if mot in index_mots:
                index = index_mots.index(mot)
                frequences[l][index] += 1

    sum_frequencies = len(mat_train[df_train['label']=='ham']) + len(mat_train[df_train['label']=='spam'])
    if sum_frequencies > 0:
        for j in range(len(frequences[l])):
            frequences[l][j] /= sum_frequencies


In [28]:
# Affichage de quelques valeurs de fréquence non nulles 
print("Seuls les 10 premiers non nuls sont affichés.")        
for l in frequences:
    nb= 0
    print("Pour le label",l, ":")
    for j in range(0,len(frequences[l])):
        if frequences[l][j] != 0:
            if (nb < 10):
                print(f'\t {index_mots[j]}:\t {frequences[l][j]:0.6f}')
            nb +=1

Seuls les 10 premiers non nuls sont affichés.
Pour le label ham :
	 ':	 0.003597
	 't:	 0.003597
	 1:	 0.014388
	 100:	 0.003597
	 1st:	 0.003597
	 2:	 0.050360
	 2marrow:	 0.003597
	 2nd:	 0.003597
	 3:	 0.017986
	 30:	 0.003597
Pour le label spam :
	 0:	 0.003597
	 00:	 0.007194
	 01223585334:	 0.003597
	 0207:	 0.003597
	 03:	 0.003597
	 07801543489:	 0.003597
	 0800:	 0.003597
	 08000930705:	 0.007194
	 08000938767:	 0.003597
	 08002888812:	 0.003597


In [29]:
print("Frequences max:")
print("\tpour ham:", max(frequences['ham']), "pour le mot",index_mots[np.argmax(frequences['ham'])])
print("\tpour spam:", max(frequences['spam']), "pour le mot",index_mots[np.argmax(frequences['spam'])])


Frequences max:
	pour ham: 0.1618705035971223 pour le mot u
	pour spam: 0.06474820143884892 pour le mot call


## Classification de textes

<div class="alert alert-block alert-warning">
On considère deux variables $X$ et $Y$ :
    <ul>
        <li>$X$ est un message</li>
        <li>$Y$ est le label d'un message et peut prendre 2 valeurs : <code>'ham'</code> et <code>'spam'</code></li>
    </ul>

Avec les fonctions précédentes on peut représenter les messages d'un corpus de documents sous la forme d'un vecteur $X \in \{0, 1\}^p$ où $p$ est le nombre total de mots de l'index. Le $i$ème message est représenté par le vecteur ${\bf x}_i = [x_{i1} \dots x_{ip}]$, où $x_{ij}$ vaut 1 si le $j$ème mot de l'index est présent dans le message $i$, et 0 sinon.
    
Comme vu en cours, pour un classifieur bayésien, nous devons estimer, à partir de la base <code>df_train</code>, les probabilités $p(ham)$, $p(spam)$, $p(X |ham)$ et $p(X | spam)$.

Les 2 premières sont simples à calculer : on compte le nombre de fois où le label apparaît parmi les exemples.

Pour un label $Y$ et un exemple ${\bf x}$, le calcul de $p({\bf x} | Y)$ se fait en utilisant les probabilités $p(mot | Y)$ de tous les mots qui composent ${\bf x}$ (et qui sont des mots de l'index des mots). $p(mot | Y)$ est la probabilité que le mot <code>mot</code> apparaisse dans un message sachant que ce message appartient à la classe $Y$.
    
On pose ainsi

$$ p({\bf x} | Y) = \prod_{mot \in index\_mots} p(mot | Y)^{x_{mot}} \left(1 - p(mot | Y)\right)^{1 - x_{mot}} $$

où $x_{mot}$ correspond à $1$ ou $0$ selon que le `mot` apparaît dans ${\bf x}$ ou pas. Ce terme permet de retenir soit la probabilité $p(mot | Y)$ si le mot est dans ${\bf x}$, soit la probabilité qu'il n'y soit pas ($1-p(mot |Y)$).
    
Une fois que $p({\bf x} | Y)$ est calculée, on peut estimer $p(Y|X)$ grâce au théorème de Bayes (cf. cours 6):

$$p(Y|{\bf x}) = p(Y) p({\bf x} | Y)$$

avec $Y$ qui vaut soit 'ham', soit 'spam'.    

    
Une fois $p(Y{{\bf x}})$ calculée pour chaque valeur de label, pour prédire le label de ${\bf x}$ on choisit le label qui possède la plus forte probabilité.
</div>

In [30]:
for l in ['ham', 'spam']:
    proba = len(df_train[df_train['label'] == l]) / len(df_train)
    print(f'p({l:4}) = {proba:0.4f}')


p(ham ) = 0.8669
p(spam) = 0.1331


<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `proba_mot` qui étant donné un mot, un label, une liste de mots, et un dictionnaire avec les fréquences des mots par label (comme `frequences` et compatible avec la liste de mots) rend $p(mot| label)$ la probabilité du mot d'appartenir au label donné.


In [31]:
def proba_mot(mot, label, index_mots, frequences):
    """
    Calcule la probabilité p(mot | label), c'est-à-dire la probabilité qu'un mot apparaisse 
    dans un message sachant qu'il appartient à une classe donnée.

    Arguments :
    mot         -- (str) Le mot dont on veut estimer la probabilité conditionnelle
    label       -- (str) Le label ('ham' ou 'spam')
    index_mots  -- (list) Liste des mots de l'index
    frequences  -- (dict) Dictionnaire des fréquences des mots par label, stockées sous forme de liste

    Retour :
    float -- La probabilité p(mot | label)
    """
    if label not in frequences:
        return 0.0
    
    if mot in index_mots:
        index = index_mots.index(mot)  # Trouver l'index du mot dans la liste index_mots
        return frequences[label][index]  # Accéder à la probabilité correspondante dans la liste
    else:
        return 0.0

# Exemple de test
try:
    print(proba_mot("visit", "cookie", index_mots, frequences))  # Devrait lever une erreur
except ValueError as e:
    print(e)  # Affiche le message d'erreur

0.0


In [32]:
# probabilité d'un mot qui n'est pas dans l'index:
proba_mot("toto", "ham", index_mots, frequences)

0.0

In [33]:
# probabilité d'un mot de l'index pour un label qui n'existe pas:
proba_mot("visit", "cookie", index_mots, frequences)

0.0

In [34]:
# probabilité d'un mot de l'index pour un label qui existe:
proba_mot("call", "ham", index_mots, frequences)

0.046762589928057555

In [35]:
# probabilité d'un mot de l'index pour un label qui existe:
proba_mot("call", "spam", index_mots, frequences)

0.06474820143884892

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `proba_exemple` qui étant donné un exemple représenté sous la forme d'une liste de mots, un label, une liste de mots, et un dictionnaire comme `frequences` (compatible avec la liste de mots) rend $p(exemple|label)$ la probabilité de l'exemple d'appartenir au label.


In [36]:
def proba_exemple(exemple, label, index_mots, frequences):
    """
    Calcule la probabilité p(exemple | label), c'est-à-dire la probabilité qu'un message 
    appartienne à une classe donnée en fonction des mots qu'il contient.

    Arguments :
    exemple     -- (list) Liste des mots du message
    label       -- (str) Le label ('ham' ou 'spam')
    index_mots  -- (list) Liste des mots de l'index
    frequences  -- (dict) Dictionnaire des fréquences des mots par label

    Retour :
    float -- La probabilité p(exemple | label)
    """
    if label not in frequences:
        raise ValueError(f"Label inconnu: {label}. Labels possibles: {list(frequences.keys())}")

    prob = 1.0  # On initialise la probabilité totale à 1
    for mot in index_mots:  # Parcours de tous les mots de l'index
        p_mot_label = proba_mot(mot, label, index_mots, frequences)  # Probabilité du mot pour ce label
        
        if mot in exemple:  # Si le mot est présent dans l'exemple
            prob *= p_mot_label if p_mot_label > 0 else 1e-10  # Évite les zéros (lissage)
        else:  # Si le mot n'est pas dans l'exemple
            prob *= (1 - p_mot_label) if (1 - p_mot_label) > 0 else 1e-10  # Évite les zéros

    return prob

In [37]:
proba_exemple(df_train['les_mots'].iloc[10], 'ham', index_mots,frequences)

2.1316038568344261e-16

In [38]:
for i in range(0,len(df_train)):
    p_spam = proba_exemple(df_train['les_mots'].iloc[i], 'spam', index_mots,frequences)
    p_ham = proba_exemple(df_train['les_mots'].iloc[i], 'ham', index_mots,frequences)
    if (p_spam>0) and (p_ham>0):
        print("Exemple",i,": p(ham)=",p_ham,"\tp(spam)=",p_spam)


Exemple 0 : p(ham)= 1.164955756737393e-23 	p(spam)= 1.4595179541817965e-79
Exemple 1 : p(ham)= 1.5565002136037318e-22 	p(spam)= 1.1721703626275303e-85
Exemple 2 : p(ham)= 1.0781992278896756e-10 	p(spam)= 3.188472652825244e-34
Exemple 3 : p(ham)= 2.097655209063231e-29 	p(spam)= 7.968968029832606e-98
Exemple 4 : p(ham)= 7.642382973891961e-14 	p(spam)= 6.400050179946332e-44
Exemple 5 : p(ham)= 1.0296306439439477e-33 	p(spam)= 1.9482505694836607e-123
Exemple 6 : p(ham)= 1.924376175409075e-55 	p(spam)= 3.3574401069433055e-212
Exemple 7 : p(ham)= 2.099159632352872e-30 	p(spam)= 6.4000501799463174e-114
Exemple 8 : p(ham)= 2.6297307381977156e-08 	p(spam)= 3.188472652825249e-14
Exemple 9 : p(ham)= 3.9066589966635926e-17 	p(spam)= 7.033395557702747e-56
Exemple 10 : p(ham)= 2.1316038568344261e-16 	p(spam)= 1.1510731598647096e-46
Exemple 11 : p(ham)= 1.7725154407193735e-09 	p(spam)= 3.1884726528252445e-24
Exemple 12 : p(ham)= 3.212123616578139e-68 	p(spam)= 1.2076488517413165e-254
Exemple 13 : p(h

<font color="RED" size="+1"><b>[Q]</b></font> À partir de `mat_train` et de `frequences` et en utilisant des opérations matricielles, on peut aussi calculer les probabilités pour chaque label de tous les exemples. En terme de temps de calcul, cela peut être plus efficace.

Donner l'instruction permettant de calculer toutes ces probabilités, puis vérifier que vous trouver les mêmes valeurs pour les exemples précédents.


In [39]:
toutes_probas = dict()
for l in les_labels:
    # Vérifier si `frequences[l]` est un dict ou une liste
    if isinstance(frequences[l], dict):
        p_mots = np.array([frequences[l].get(mot, 1e-10) for mot in index_mots])  # Cas dict
    else:
        p_mots = np.array(frequences[l])  # Cas liste, on suppose que l'ordre correspond à `index_mots`

    # Éviter log(0) et log(1)
    p_mots = np.clip(p_mots, 1e-10, 1 - 1e-10)

    # Calcul de log(p(X | Y)) pour chaque exemple
    log_probas = (mat_train @ np.log(p_mots)) + ((1 - mat_train) @ np.log(1 - p_mots))

    # Conversion en probabilité avec exponentielle
    toutes_probas[l] = np.exp(log_probas)

In [40]:
for i in range(0,len(df_train)):
    p_spam = toutes_probas['spam'][i]
    p_ham = toutes_probas['ham'][i]
    if (p_spam>0) and (p_ham>0):
        print("Exemple",i,": p(ham)=",p_ham,"\tp(spam)=",p_spam)



Exemple 0 : p(ham)= 1.1649557189928175e-23 	p(spam)= 1.459517817279011e-79
Exemple 1 : p(ham)= 1.5565001631731123e-22 	p(spam)= 1.1721702527951879e-85
Exemple 2 : p(ham)= 1.0781991929560152e-10 	p(spam)= 3.1884723524711113e-34
Exemple 3 : p(ham)= 2.097655141099189e-29 	p(spam)= 7.968967283937017e-98
Exemple 4 : p(ham)= 7.64238272627869e-14 	p(spam)= 6.400049577701555e-44
Exemple 5 : p(ham)= 1.0296306105839059e-33 	p(spam)= 1.948250387711854e-123
Exemple 6 : p(ham)= 1.9243761130592766e-55 	p(spam)= 3.3574397963799436e-212
Exemple 7 : p(ham)= 2.0991595643400722e-30 	p(spam)= 6.4000495821814024e-114
Exemple 8 : p(ham)= 2.6297306529944177e-08 	p(spam)= 3.188472351833408e-14
Exemple 9 : p(ham)= 3.9066588700877974e-17 	p(spam)= 7.033394896563536e-56
Exemple 10 : p(ham)= 2.1316037877704464e-16 	p(spam)= 1.1510730515487138e-46
Exemple 11 : p(ham)= 1.7725153832898634e-09 	p(spam)= 3.188472352152248e-24
Exemple 12 : p(ham)= 3.2121235125052324e-68 	p(spam)= 1.2076487405169495e-254
Exemple 13 : p(

<font color="RED" size="+1"><b>[Q]</b></font> En utilisant les fonctions écrites, calculer le taux de bonne prédiction du classifieur bayésien naïf pour chaque valeur de label pour le dataset d'apprentissage. 

*Remarque*: une solution efficace peut ne pas utiliser toutes les fonctions précédentes...

In [41]:
# Initialisation des taux et des probabilités
taux = dict()
toutes_probas = dict()
for l in les_labels:
    taux[l] = {True: 0, False: 0}
    toutes_probas[l] = np.zeros(len(df_train))  # Stocke p(X | label) pour chaque exemple

tic = time.time()

# Étape 1 : Calcul de p(X | label) pour chaque label
for l in les_labels:
    if isinstance(frequences[l], dict):
        p_mots = np.array([frequences[l].get(mot, 1e-10) for mot in index_mots])
    else:
        p_mots = np.array(frequences[l])

    p_mots = np.clip(p_mots, 1e-10, 1 - 1e-10)

    log_probas = (mat_train @ np.log(p_mots)) + ((1 - mat_train) @ np.log(1 - p_mots))
    toutes_probas[l] = np.exp(log_probas)  # p(X | label)

# Étape 2 : Calcul de p(label | X) en utilisant Bayes
probas_labels = {l: sum(df_train["label"] == l) / len(df_train) for l in les_labels}  # p(label)
posterior = {l: toutes_probas[l] * probas_labels[l] for l in les_labels}  # p(X | label) * p(label)

# Normalisation pour obtenir des probabilités conditionnelles p(label | X)
somme_probas = sum(posterior.values())
for l in les_labels:
    posterior[l] /= somme_probas

# Étape 3 : Prédiction et comptage des bonnes classifications
for i in range(len(df_train)):
    prediction = max(les_labels, key=lambda l: posterior[l][i])  # Label avec la plus grande proba
    vrai_label = df_train.iloc[i]["label"]  # Vrai label de l'exemple

    if prediction == vrai_label:
        taux[vrai_label][True] += 1
    else:
        taux[vrai_label][False] += 1

toc = time.time()

print("Résultats des classements : ", taux)
print("Temps: ", toc - tic)

# Étape 4 : Calcul du taux de bonne classification
taux_classification = {l: taux[l][True] / (taux[l][True] + taux[l][False]) for l in les_labels}
print("Taux de bonne classification : ", taux_classification)

Résultats des classements :  {'ham': {True: 239, False: 2}, 'spam': {True: 37, False: 0}}
Temps:  0.005668163299560547
Taux de bonne classification :  {'ham': 0.991701244813278, 'spam': 1.0}


<font color="RED" size="+1"><b>[Q]</b></font> Même question pour calculer le taux de bonne classification pour le dataset de test.

*Remarque*: L'index des mots ainsi que la matrice des fréquences restent les mêmes, ce sont toujours celles construites à partir de la base d'apprentissage. Par contre, la matrice des présences doit, elle, être recalculées.

In [42]:
# Étape 1 : Construire `mat_test`
def construire_matrice_presence(df, index_mots):
    """Construit la matrice de présence des mots pour un dataset donné"""
    mat_presence = np.zeros((len(df), len(index_mots)))
    for i, message in enumerate(df["message"]):
        mots_message = set(message.split())  # Tokenisation simple
        for j, mot in enumerate(index_mots):
            if mot in mots_message:
                mat_presence[i, j] = 1  # Présence du mot
    return mat_presence

mat_test = construire_matrice_presence(df_test, index_mots)

# Étape 2 : Initialisation des taux et des probabilités
taux = {l: {True: 0, False: 0} for l in les_labels}
toutes_probas = {l: np.zeros(len(df_test)) for l in les_labels}

tic = time.time()

# Étape 3 : Calcul de p(X | label) pour chaque label
for l in les_labels:
    if isinstance(frequences[l], dict):
        p_mots = np.array([frequences[l].get(mot, 1e-10) for mot in index_mots])
    else:
        p_mots = np.array(frequences[l])

    p_mots = np.clip(p_mots, 1e-10, 1 - 1e-10)

    log_probas = (mat_test @ np.log(p_mots)) + ((1 - mat_test) @ np.log(1 - p_mots))
    toutes_probas[l] = np.exp(log_probas)  # p(X | label)

# Étape 4 : Calcul de p(label | X) en utilisant Bayes
probas_labels = {l: sum(df_train["label"] == l) / len(df_train) for l in les_labels}  # p(label)
posterior = {l: toutes_probas[l] * probas_labels[l] for l in les_labels}  # p(X | label) * p(label)

# Normalisation pour obtenir des probabilités conditionnelles p(label | X)
somme_probas = sum(posterior.values())
for l in les_labels:
    posterior[l] /= somme_probas

# Étape 5 : Prédiction et comptage des bonnes classifications
for i in range(len(df_test)):
    prediction = max(les_labels, key=lambda l: posterior[l][i])  # Label avec la plus grande proba
    vrai_label = df_test.iloc[i]["label"]  # Vrai label de l'exemple

    if prediction == vrai_label:
        taux[vrai_label][True] += 1
    else:
        taux[vrai_label][False] += 1

toc = time.time()

print("Résultats des classements : ", taux)
print("Temps: ", toc - tic)

# Étape 6 : Calcul du taux de bonne classification
taux_classification = {l: taux[l][True] / (taux[l][True] + taux[l][False]) for l in les_labels}
print("Taux de bonne classification : ", taux_classification)

Résultats des classements :  {'ham': {True: 3504, False: 1080}, 'spam': {True: 569, False: 141}}
Temps:  0.07705807685852051
Taux de bonne classification :  {'ham': 0.7643979057591623, 'spam': 0.8014084507042254}


## Evaluation du classifieur Naive Bayes

Pour tout ce que l'on a fait jusque-là, on a travaillé sur tout le dataset, pour pouvoir l'évaluer il est nécessaire de le séparer en ensemble d'apprentissage et ensemble de test.

<font color="RED" size="+1"><b>[Q]</b></font> Découper aléatoirement `df_spam` en 2 parties égales, chacune contenant des exemples des 2 labels, avec la même distribution des labels dans chaque partie. 
Une des parties sert à apprendre l'index des mots et leurs fréquences.
L'autre partie n'est utilisée que pour calculer le taux de bonne classification par label.

Donner ensuite les taux de bonne classification par label pour l'ensemble de train et pour l'ensemble de test.

*Remarque*: certains mots de la partie de test pourront ne pas être présents dans l'index de mots car ils peuvent être absents de la partie d'apprentissage.

In [ ]:
from sklearn.model_selection import train_test_split

# Séparation en train/test avec conservation des proportions de labels
df_train, df_test = train_test_split(df_spam, test_size=0.5, stratify=df_spam["label"], random_state=42)

# Vérification des distributions
print(df_train["label"].value_counts(normalize=True))
print(df_test["label"].value_counts(normalize=True))

<font color="RED" size="+1"><b>[Q]</b></font> Mettre en place une approche par validation croisée pour évaluer le taux de bonne classification moyen de cette approche. 

<font color="RED" size="+1"><b>[Q]</b></font> Comparer les résultats obtenus (taux de bonne classification avec la validation croisée, temps de calcul) avec ceux obtenus avec l'application d'un classifieur par $k$ plus proches voisins. Pour cela, la `mat_spam` doit être utilisée comme description des données. 